# Sensors Analytics REST API

## Deploy the API
- Go to the *Machine* tab, then set *Incoming connections* to **ON**. The API will be accessible through the indicated tunnelling link.  
- Run the notebook.



## Import the required modules

In [1]:
import json
import redis
import time
from datetime import datetime
import paho.mqtt.client as mqtt

## Connect to the Redis Database

In [2]:
REDIS_HOST="redis-12999.c135.eu-central-1-1.ec2.redns.redis-cloud.com"
REDIS_PORT="12999"
REDIS_USERNAME='default'
REDIS_PASSWORD='LHz8VkDlHWjkAFh0jVh6WsyMQEe8c49D'

redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USERNAME, password=REDIS_PASSWORD)

# Test Redis connection
try:
    response = redis_client.ping()
    print("Connected to Redis:", response)
except Exception as e:
    print("Redis connection failed:", e)
    exit(1)

Connected to Redis: True


In [3]:
def create_timeseries_if_not_exists(redis_client, key, retention_msecs=30 * 24 * 60 * 60 * 1000):
    try:
        redis_client.ts().create(
            key,
            retention_msecs=retention_msecs,
            chunk_size=128,
            duplicate_policy="last"
        )
        print(f"Created TimeSeries {key}")
    except redis.ResponseError as e:
        if "already exists" in str(e):
            print(f"TimeSeries {key} already exists")
        else:
            raise

# MQTT Callbacks
def on_connect(client, userdata, flags, rc):
    print(f"Connected to MQTT broker with result code: {rc}")
    client.subscribe(MQTT_TOPIC)
    print(f"Subscribed to topic: {MQTT_TOPIC}")

def on_message(client, userdata, msg):
    try:
        # Parse the JSON message
        data = json.loads(msg.payload.decode())
        mac_address = data["mac_address"]
        timestamp_ms = data["timestamp"]
        temperature = data["temperature"]
        humidity = data["humidity"]
        
        # Store in Redis TimeSeries
        temp_key = f"{mac_address}:temperature"
        humid_key = f"{mac_address}:humidity"
        
        # # Make sure TimeSeries exist
        # create_timeseries_if_not_exists(redis_client, temp_key)
        # create_timeseries_if_not_exists(redis_client, humid_key)
        
        # Add data points
        redis_client.ts().add(temp_key, timestamp_ms, temperature)
        redis_client.ts().add(humid_key, timestamp_ms, humidity)
        
        # Print confirmation
        print(f"{datetime.fromtimestamp(timestamp_ms/1000)}: Stored T:{temperature}°C, H:{humidity}%")
        
    except json.JSONDecodeError as e:
        print(f"Error decoding message: {e}")
    except Exception as e:
        print(f"Error processing message: {e}")


## MQTT Broker

In [4]:
MQTT_BROKER = "mqtt.eclipseprojects.io"
MQTT_PORT = 1883
MQTT_TOPIC = "s334033"

# Setup MQTT client
mqtt_client = mqtt.Client()
mqtt_client.on_connect = on_connect
mqtt_client.on_message = on_message

# Connect to MQTT broker
try:
    mqtt_client.connect(MQTT_BROKER, MQTT_PORT, 60)
    print(f"Starting MQTT loop... Listening on topic {MQTT_TOPIC}")
    mqtt_client.loop_forever()
except KeyboardInterrupt:
    print("Stopping...")
    mqtt_client.disconnect()
except Exception as e:
    print(f"MQTT connection failed: {e}")
    mqtt_client.disconnect()

/tmp/ipykernel_172/4270315520.py:6: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  mqtt_client = mqtt.Client()
Starting MQTT loop... Listening on topic s334033
Connected to MQTT broker with result code: 0
Subscribed to topic: s334033
2025-01-27 13:58:21.579000: Stored T:21°C, H:45%
2025-01-27 13:58:23.834000: Stored T:21°C, H:45%
2025-01-27 13:58:26.088000: Stored T:21°C, H:45%
2025-01-27 13:58:28.343000: Stored T:21°C, H:45%
2025-01-27 13:58:30.597000: Stored T:21°C, H:45%
2025-01-27 13:58:32.852000: Stored T:21°C, H:45%
2025-01-27 13:58:35.106000: Stored T:21°C, H:45%
Stopping...


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5772165a-963f-4d38-9b9d-11ecac130c62' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>